# 5.3.4 - SQL with Spark

In [3]:
import pyspark
from pyspark.sql import SparkSession, types

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/02 20:26:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


After importing the basics and setting up the session, load in taxi data:

In [7]:
df_green = spark.read.parquet('data/pq/green/*/*/')
df_yellow = spark.read.parquet('data/pq/yellow/*/*/')

In [13]:
df_yellow.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



Want to run a transformation that works on both datasets, so we'll merge these below on the fields that match. We'll have to rename one or both sets so that the date column matches in name.

In [14]:
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [15]:
set(df_green.columns) & set(df_yellow.columns)

{'DOLocationID',
 'PULocationID',
 'RatecodeID',
 'VendorID',
 'congestion_surcharge',
 'dropoff_datetime',
 'extra',
 'fare_amount',
 'improvement_surcharge',
 'mta_tax',
 'passenger_count',
 'payment_type',
 'pickup_datetime',
 'store_and_fwd_flag',
 'tip_amount',
 'tolls_amount',
 'total_amount',
 'trip_distance'}

We won't use this set exactly, as we want to preserve the column order

In [16]:
common_columns = []
for column in df_green.columns:
    if column in df_yellow.columns:
        common_columns.append(column)

common_columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge']

We'll append a `service_type` column to ensure we know where the data is coming from in each case. We'll need to use the `literal` function for god knows what reason in this case.

In [18]:
from pyspark.sql import functions as F

df_green_sel = df_green \
    .select(common_columns) \
    .withColumn('service_type', F.lit('green'))

df_yellow_sel = df_yellow \
    .select(common_columns) \
    .withColumn('service_type', F.lit('yellow'))

In [19]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [21]:
df_trips_data.groupBy('Service_type').count().show()

+------------+--------+
|Service_type|   count|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



We haven't *really* used SQL so far- from here we can give it a go on the combined dataset. We need to register the df as a table to start.

In [22]:
df_trips_data.registerTempTable('trips_data')

/home/sam/anaconda3/envs/pyspark/lib/python3.9/site-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [27]:
spark.sql("""
    SELECT
        service_type,
        COUNT(*) AS records_count
    FROM trips_data
    GROUP BY service_type
    """).show()

+------------+-------------+
|service_type|records_count|
+------------+-------------+
|       green|      2304517|
|      yellow|     39649199|
+------------+-------------+



Now going to execute a query from week 4: [dm_monthly_zone_revenue.sql](https://github.com/DataTalksClub/data-engineering-zoomcamp/blob/main/week_4_analytics_engineering/taxi_rides_ny/models/core/dm_monthly_zone_revenue.sql)

In [29]:
df_trips_data.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge',
 'service_type']

In [34]:
df_result = spark.sql("""
    SELECT 
        -- Revenue grouping 
        PULocationID AS revenue_zone,
        DATE_TRUNC('month', pickup_datetime) AS revenue_month, 

        service_type, 

        -- Revenue calculation 
        SUM(fare_amount) AS revenue_monthly_fare,
        SUM(extra) AS revenue_monthly_extra,
        SUM(mta_tax) AS revenue_monthly_mta_tax,
        SUM(tip_amount) AS revenue_monthly_tip_amount,
        SUM(tolls_amount) AS revenue_monthly_tolls_amount,
        SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
        SUM(total_amount) AS revenue_monthly_total_amount,
        SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

        -- Additional calculations
        COUNT(*) AS total_monthly_trips,
        AVG(passenger_count) AS avg_montly_passenger_count,
        AVG(trip_distance) AS avg_montly_trip_distance

    FROM trips_data
    GROUP BY
        revenue_zone,
        revenue_month,
        service_type
    ORDER BY
        revenue_zone,
        revenue_month,
        service_type
""")

In [35]:
df_result.show()

+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+-------------------+--------------------------+------------------------+
|revenue_zone|      revenue_month|service_type|revenue_monthly_fare|revenue_monthly_extra|revenue_monthly_mta_tax|revenue_monthly_tip_amount|revenue_monthly_tolls_amount|revenue_monthly_improvement_surcharge|revenue_monthly_total_amount|revenue_monthly_congestion_surcharge|total_monthly_trips|avg_montly_passenger_count|avg_montly_trip_distance|
+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+-------------------+------------

In [37]:
df_result.write.parquet('./data/report/revenue')

In [38]:
!ls -lh ./data/report/revenue

total 524K
-rw-r--r-- 1 sam sam    0 Mar  2 21:14 _SUCCESS
-rw-r--r-- 1 sam sam 522K Mar  2 21:14 part-00000-e7e14d87-9837-4b65-b646-46955c7141c8-c000.snappy.parquet


In this case, there's only one file- if there are multiple redundant files, we could use the `coalesce()` function to reduce the number of files produced:

```py
df_result \
    .coalesce(1) \
    .write \
    .parquet('./data/report/revenue', mode='overwrite')
```